# Word Embedding Visualizations.

Name: Anthony Kevin Oktavius

Here, we'll experiment with various embedding layer, with these following techniques.

* TF-IDF
* Count Vectorizer
* Word2Vec
* Neural network embedding layer.

As time passes, I'll try to add various  techniques. We also use dimensionality reduction technique to reduce the dimension of each layers, so we can visualize the relationshop between each words with matplotlib.

Library Import

In [1]:
import numpy as np
import zipfile
import gc
import pickle
import cv2
import math
import warnings
import random
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import albumentations as A
import torchvision
import torchvision.transforms.functional as TF
import torchvision.transforms as transforms
import torch.utils.data as data_utils
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import torch.nn as nn
# import torchmetrics
import torch.nn.functional as F
import PIL
import torch.utils.data as data_utils
import pickle
import collections

# import pytorch_lightning as pl
import imutils
import zipfile

from wordcloud import WordCloud
from PIL import Image
from torch.utils.data import Dataset
from collections import defaultdict
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.python.client import device_lib
from zipfile import ZipFile
from IPython import display
from torchvision import models, transforms
from google.colab.patches import cv2_imshow
from sklearn.metrics import confusion_matrix, roc_curve
from sklearn.feature_extraction.text import CountVectorizer
from google.colab.patches import cv2_imshow

Dataset Loading with OpinRank dataset.

In [2]:
! wget https://www.cs.cmu.edu/~./enron/enron_mail_20150507.tar.gz

--2023-09-28 14:47:35--  https://www.cs.cmu.edu/~./enron/enron_mail_20150507.tar.gz
Resolving www.cs.cmu.edu (www.cs.cmu.edu)... 128.2.42.95
Connecting to www.cs.cmu.edu (www.cs.cmu.edu)|128.2.42.95|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 443254787 (423M) [application/x-gzip]
Saving to: ‘enron_mail_20150507.tar.gz’

enron_mail_20150507 100%[===================>] 422.72M  1.57MB/s    in 4m 34s  

2023-09-28 14:52:10 (1.54 MB/s) - ‘enron_mail_20150507.tar.gz’ saved [443254787/443254787]



Unzip the dataset in tar file format.

In [3]:
! tar -xzf /content/enron_mail_20150507.tar.gz

## Data Preparation

Gather all of the email paths from all-documents folder.

In [4]:
root_path = "maildir/"
email_paths_from_all_documents_folder = glob.glob(os.path.join(root_path, "*/all_documents/*."))
print(email_paths_from_all_documents_folder[:10])

['maildir/whalley-g/all_documents/54.', 'maildir/whalley-g/all_documents/226.', 'maildir/whalley-g/all_documents/344.', 'maildir/whalley-g/all_documents/302.', 'maildir/whalley-g/all_documents/299.', 'maildir/whalley-g/all_documents/64.', 'maildir/whalley-g/all_documents/193.', 'maildir/whalley-g/all_documents/103.', 'maildir/whalley-g/all_documents/104.', 'maildir/whalley-g/all_documents/358.']


Get body email data, after X-Filename.

In [67]:
print("Sample Email data: ")

def read_file(path):
  return open(path, "r", errors='ignore')

def get_removed_components_email_text(text):
  texts_per_line = text.split("\n")

  full_email_text_per_line = []

  email_components = [
    "Message-ID: ", "Date: ", "From: ",
    "To: ", "Subject: ", "Mime-Version: ",
    "Content-Type: ", "Content-Transfer-Encoding: ", "X-From: ",
    "X-cc: ", "X-bcc: ", "X-Folder: ", "X-FileName: ",
    "X-Origin: ", "X-Filename: ", "cc: ", "@enron.com",
  ]

  for text_per_line in text.split("\n"):
    is_exist_component = False
    for component in email_components:
      if text_per_line.find(component) != -1:
        is_exist_component = True
        break
    if is_exist_component == False:
      full_email_text_per_line.append(text_per_line)

  return " ".join(full_email_text_per_line)

sample_file = read_file(email_paths_from_all_documents_folder[0])
sample_body_email_data = get_removed_components_email_text(sample_file.read())

print(sample_body_email_data)

Sample Email data: 
 Following please find the Daily EnronOnline Executive Summary.  EnronOnline Executive Summary for 12/04/00 Transaction Summary	 External Transactions Today	5,500 Average Daily External Transactions (30 day Trailing Avg)	4,097 Total Life to Date Transactions	493,715  EnronOnline Daily % of Entity's Total Transactions for 12/04/00	Trans	Percent Enron North America	4,957	80% Enron Europe Limited	119	27% Global Products	224	34% Enron Metals	200	8%  EnronOnline Percentage of Total Corporate Transactions  (30 day Trailing Avg)	54%  Gross Notional Value of EnronOnline Transactions	Total Today	6,718,666,961 MTD	11,437,759,254 LTD	290,608,284,119 Average Daily Notional Value (30 Day Trailing Avg)	3,095,165,757 	 Notional Revenue from Physical Flows to Date	19,315,426,224  Counterparties, Users, and Products Offered	 Counterparties having transacted to date	593 New Counterparties transacting for the first time today	2 Number of Products Offered Today	1,138 Total Number of Ex

Putting it all together

In [69]:
def get_dataset():
  root_path = "maildir/"
  email_paths_from_all_documents_folder = glob.glob(os.path.join(root_path, "*/all_documents/*."))

  dataset = []

  for email_path in email_paths_from_all_documents_folder:
    file_ = read_file(email_path)
    email_body = get_removed_components_email_text(file_.read())
    dataset.append(email_body)

  return dataset

dataset = get_dataset()
print(f"Number of data: {len(dataset)}")
dataset[:10]

Number of data: 128103


[" Following please find the Daily EnronOnline Executive Summary.  EnronOnline Executive Summary for 12/04/00 Transaction Summary\t External Transactions Today\t5,500 Average Daily External Transactions (30 day Trailing Avg)\t4,097 Total Life to Date Transactions\t493,715  EnronOnline Daily % of Entity's Total Transactions for 12/04/00\tTrans\tPercent Enron North America\t4,957\t80% Enron Europe Limited\t119\t27% Global Products\t224\t34% Enron Metals\t200\t8%  EnronOnline Percentage of Total Corporate Transactions  (30 day Trailing Avg)\t54%  Gross Notional Value of EnronOnline Transactions\tTotal Today\t6,718,666,961 MTD\t11,437,759,254 LTD\t290,608,284,119 Average Daily Notional Value (30 Day Trailing Avg)\t3,095,165,757 \t Notional Revenue from Physical Flows to Date\t19,315,426,224  Counterparties, Users, and Products Offered\t Counterparties having transacted to date\t593 New Counterparties transacting for the first time today\t2 Number of Products Offered Today\t1,138 Total Numb

Performing Word Cloud.